[과제] train.csv 파일을 불러와서 분류모델링 평가를 아래와 같이 수행하세요.
- dt 모델을 적용하고 GridSearchCV를 사용, 교차검증과 성능 개선을 위한 하이퍼파라미터 튜닝
- accuracy에 추가하여 confusion matrix, accuracy, precision, recall, roc auc score 평가


In [ ]:
tdf = pd.read_csv('./dataset/train.csv')
y_tdf = tdf['Survived']
X_tdf = tdf.drop('Survived',axis = 1)
X_tdf = transform_features(X_tdf)
X_tdf.head()

In [ ]:
# dt 모델을 적용하고 GridSearchCV를 사용, 교차검증과 성능 개선을 위한 하이퍼파라미터 튜닝
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

X_train,X_test, y_train, y_test = train_test_split(X_tdf,y_tdf,test_size = 0.2,random_state = 11)

dt_clf = DecisionTreeClassifier(random_state = 11)
dt_clf.fit(X_train,y_train)
dt_pred = dt_clf.predict(X_test)
print('dt_clf',accuracy_score(y_test,dt_pred))

In [ ]:
from sklearn.model_selection import GridSearchCV

parameters = {'max_depth':[2,3,5,10],'min_samples_split':[2,3,5],'min_samples_leaf':[1,5,8]}

grid_dclf = GridSearchCV(dt_clf,param_grid=parameters,scoring='accuracy',cv=5)
grid_dclf.fit(X_train,y_train)

print(grid_dclf.best_params_)
print(grid_dclf.best_score_)
best_dclf = grid_dclf.best_estimator_

dpredictions = best_dclf.predict(X_test)
accuracy = accuracy_score(y_test,dpredictions)

print(accuracy)

In [ ]:
# accuracy에 추가하여 confusion matrix, accuracy, precision, recall, roc auc score 평가
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
pred_proba = dt_clf.predict_proba(X_test)[:,1]

def get_clf_evals(y_test,pred=None,pred_proba=None):
    confusion = confusion_matrix(y_test,pred)
    accuracy = accuracy_score(y_test,pred)
    precision = precision_score(y_test,pred)
    recall = recall_score(y_test,pred)
    f1 = f1_score(y_test,pred)
    roc_auc = roc_auc_score(y_test,pred_proba)
    
    print(f'오차 행렬:\n {confusion}')
    print(f'정확도: {round(accuracy,4)}, 정밀도: {round(precision,4)}, 재현율: {round(recall,4)}, f1 score: {round(f1,4)}, auc: {round(roc_auc,4)}')
    
get_clf_evals(y_test,dt_pred,pred_proba)